## Importacion de librerias

In [5]:
# Import modules

# import urllib

import os
os.add_dll_directory(r"C:\Program Files (x86)\ZED SDK\bin")

import sys
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import pyzed.sl as sl
print("ZED SDK cargado correctamente")

ZED SDK cargado correctamente


## Inicializamos ZED

In [8]:

zed = sl.Camera()
init_params = sl.InitParameters()
init_params.set_from_svo_file("video_salmones.svo")  # ← esta es la forma correcta
init_params.svo_real_time_mode = False

status = zed.open(init_params)
print("Estado:", status)

# Configurar imagen
image = sl.Mat()

Estado: SUCCESS


## Integración YOLOv8 + OpenCV tracker (algoritmo de deteccion)

In [9]:
model = YOLO("yolov8n.pt")  #CAMBIAR A MODELO ENTRENADO PARA SALMONES
tracker = DeepSort()

C:\Users\Alonso\AppData\Roaming\Python\Python310\site-packages\deep_sort_realtime\embedder\embedder_pytorch.py:6: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


## Configurar salida de video

In [10]:
output_path = "output/seguimiento_salmones.mp4"
fps = zed.get_camera_information().camera_configuration.fps
width = zed.get_camera_information().camera_configuration.resolution.width
height = zed.get_camera_information().camera_configuration.resolution.height
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

## Bucle de Tracking

In [12]:
while True:
    if zed.grab() != sl.ERROR_CODE.SUCCESS:
        break

    zed.retrieve_image(image, sl.VIEW.LEFT)
    frame = image.get_data()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGRA2BGR)

    # 1. Ejecutar ambos modelos
    results_seg = model_seg(frame)[0]   # Resultados de segmentación
    results_pose = model_pose(frame)[0]  # Resultados de estimación de pose

    # 2. Preparar detecciones para DeepSort (usando los bounding boxes de la segmentación)
    detections = []
    # Usamos .boxes de los resultados de segmentación
    for box in results_seg.boxes: 
        x1, y1, x2, y2 = box.xyxy[0].tolist()
        conf = float(box.conf[0])
        cls = int(box.cls[0])
        # DeepSort necesita el formato: ([x1, y1, w, h], conf, cls) o ([x1, y1, x2, y2], conf, cls)
        # Tu código original usaba [x1, y1, x2, y2], así que lo mantenemos
        detections.append(([x1, y1, x2, y2], conf, cls))

    # 3. Actualizar el tracker
    tracks = tracker.update_tracks(detections, frame=frame)

    # --- Lógica de Asociación y Dibujo ---
    # Necesitamos una forma de asociar los 'tracks' con las 'poses'
    # La forma más simple es por proximidad de sus bounding boxes (IoU)
    
    # Recolecta los datos de pose para facilitar la búsqueda
    pose_boxes = results_pose.boxes.xyxy.cpu().numpy() if results_pose.keypoints is not None else []
    pose_kpts = results_pose.keypoints.data.cpu().numpy() if results_pose.keypoints is not None else []
    
    # 4. Dibujar los tracks, IDs, y las POSES asociadas
    for track in tracks:
        if not track.is_confirmed():
            continue
        
        track_id = track.track_id
        # Obtenemos el bounding box actual del tracker
        x1, y1, x2, y2 = map(int, track.to_ltrb()) 

        # Dibujar Bounding Box y ID (tu código original)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
        cv2.putText(frame, f"ID {track_id}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50, 170, 50), 2)

        # --- Asociar y Dibujar Puntos Clave (Keypoints) ---
        
        # (Implementación simple de asociación: encontrar la pose 
        # cuyo bounding box esté más cerca del centro del track_box)
        
        track_center_x = (x1 + x2) / 2
        track_center_y = (y1 + y2) / 2
        
        best_kpts = None
        min_dist = float('inf')

        for i, (p_box, kpts) in enumerate(zip(pose_boxes, pose_kpts)):
            # Centro del box de la pose detectada
            p_center_x = (p_box[0] + p_box[2]) / 2
            p_center_y = (p_box[1] + p_box[3]) / 2
            
            # Calcular distancia
            dist = np.sqrt((track_center_x - p_center_x)**2 + (track_center_y - p_center_y)**2)
            
            # Si es la más cercana y está dentro del box (para evitar malas asociaciones)
            if dist < min_dist and (p_center_x > x1 and p_center_x < x2 and p_center_y > y1 and p_center_y < y2):
                min_dist = dist
                best_kpts = kpts
        
        # Si encontramos una pose asociada, dibujamos sus puntos clave
        if best_kpts is not None:
            for (px, py, pconf) in best_kpts:
                if pconf > 0.5: # Dibujar solo puntos con confianza
                    cv2.circle(frame, (int(px), int(py)), 3, (0, 0, 255), -1) # Puntos en rojo
        
        # (Aquí también podrías añadir la lógica para dibujar la máscara de segmentación)
        # (La asociación de máscaras seguiría una lógica similar a la de las poses)

    # --- Fin de la lógica de dibujo ---

    out.write(frame)

    cv2.imshow("Seguimiento de Salmones", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


# Cierre
out.release()
zed.close()
cv2.destroyAllWindows()


0: 384x640 (no detections), 111.4ms
Speed: 33.5ms preprocess, 111.4ms inference, 7.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 68.7ms
Speed: 3.0ms preprocess, 68.7ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 63.7ms
Speed: 2.2ms preprocess, 63.7ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 65.5ms
Speed: 2.2ms preprocess, 65.5ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 64.5ms
Speed: 2.1ms preprocess, 64.5ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 69.5ms
Speed: 2.2ms preprocess, 69.5ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 70.7ms
Speed: 2.4ms preprocess, 70.7ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 64.6ms
Speed: 2.3ms preprocess, 64.6m

('64bit', 'WindowsPE')
